In [1]:
# Dependencies and Setup
import pandas as pd
import numpy as np
import os

# File to Load (Remember to Change These)
file_to_load = "Resources/purchase_data.csv"

# Read Purchasing File and store into Pandas data frame
purchase_data = pd.read_csv(file_to_load)
purchase_data.head()

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56
2,2,Ithergue48,24,Male,92,Final Critic,4.88
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27
4,4,Iskosia90,23,Male,131,Fury,1.44


In [2]:
# making the vairs 

# Player Count
totalNumPlayers = 0

# Purchasing (All)
numOfUniqueItems = 0
avgPurPrice = 0.0
totalNumPur = 0
totalRev = 0.0

# Gender Demograph | m - male, f - female, o - other
# Count
countM = 0
countF = 0
countO = 0
# Percent 
percM = 0.0
percF = 0.0
percO = 0.0
# Total Rev of puchases
tRevM = 0.0
tRevF = 0.0
tRevO = 0.0
# Average pur price
avgPurM = 0.0
avgPurF = 0.0
avgPurO = 0.0
# Total Pur Value
tPurValM = 0.0
tPurValF = 0.0
tPurValO = 0.0
# Average Purchase Total per Person by Gender
avgPurTotM = 0.0
avgPurTotF = 0.0
avgPurTotO = 0.0

In [3]:
totalNumPlayers = purchase_data["SN"].nunique()
totalNumPlayersDf = pd.DataFrame([{"Total Players" : totalNumPlayers}])
totalNumPlayersDf

,Total Players
0,576


In [4]:
numOfUniqueItems = purchase_data["Item Name"].nunique()
avgPurPrice = purchase_data['Price'].mean()
totalNumPur = purchase_data['Purchase ID'].count()
totalRev = purchase_data["Price"].sum()
PurSumDF = pd.DataFrame([{"Number of Unique Items" : numOfUniqueItems, "Average Purchase Price" : avgPurPrice, "Total Number of Purchases" : totalNumPur, "Total Revenue" : totalRev}])
PurSumDF["Average Purchase Price"] = PurSumDF["Average Purchase Price"].map("${:.2f}".format)
PurSumDF["Total Revenue"] = PurSumDF["Total Revenue"].map("${:.2f}".format)
PurSumDF

,Number of Unique Items,Average Purchase Price,Total Number of Purchases,Total Revenue
0,179,$3.05,780,$2379.77


In [5]:
gender_df = purchase_data.loc[:, ["SN", "Gender",]]
gender_df = gender_df.drop_duplicates(["SN"])
gender_df = gender_df.groupby(['Gender'], as_index=False).agg({'SN' : 'count'})
gender_df = gender_df.sort_values("SN", ascending=False).reset_index(drop=True)
# The Hard Way 
# percM = (gender_df['SN'].loc[gender_df["Gender"] == 'Male'] / totalNumPlayers * 100).reset_index(drop=True)
# percF = (gender_df['SN'].loc[gender_df["Gender"] == 'Female'] / totalNumPlayers * 100).reset_index(drop=True)
# percO = (gender_df['SN'].loc[gender_df["Gender"] == 'Other / Non-Disclosed'] / totalNumPlayers * 100).reset_index(drop=True)
# gender_percents = pd.DataFrame({'Percentage of Players' : [percM[0], percF[0], percO[0]]})
# gender_df['Percentage of Players'] = gender_percents
# gender_df
gender_df['Percentage of Players'] = (gender_df["SN"] / totalNumPlayers * 100).map("% {:.2f}".format)
gender_df = gender_df.rename(columns={"SN": "Count"})
gender_df = gender_df.set_index("Gender")
gender_df

,Count,Percentage of Players
Gender,,
Male,484,% 84.03
Female,81,% 14.06
Other / Non-Disclosed,11,% 1.91


In [6]:
# I found a better way to do this down in the Age part. I'm leaving this here to show how much progress i made 
gender_purch_df = purchase_data.copy()
gender_purch_df = gender_purch_df.drop(["Item Name" ,'Item ID' ,"Age" ,"Purchase ID"], axis=1)
gender_purch_df = gender_purch_df.groupby(['Gender'], as_index=False).agg({'SN' : 'count', 'Price' : 'sum'})

# Calc Average Purchase Price
temp_F = purchase_data.loc[purchase_data['Gender'] == 'Female'].mean()
avgPurF = temp_F['Price']
temp_M = purchase_data.loc[purchase_data['Gender'] == 'Male'].mean()
avgPurM = temp_M['Price']
temp_O = purchase_data.loc[purchase_data['Gender'] == 'Other / Non-Disclosed'].mean()
avgPurO = temp_O['Price']
AvgPurPriceDict = pd.DataFrame({'Average Purchase Price' : [avgPurF, avgPurM, avgPurO]})
gender_purch_df["Average Purchase Price"] = AvgPurPriceDict

# Calc Total Purchase cost
temp_F = purchase_data.loc[purchase_data['Gender'] == 'Female'].sum()
tRevF = temp_F['Price']
temp_M = purchase_data.loc[purchase_data['Gender'] == 'Male'].sum()
tRevM = temp_M['Price']
temp_O = purchase_data.loc[purchase_data['Gender'] == 'Other / Non-Disclosed'].sum()
tRevO = temp_O['Price']
TotalPurValGen = pd.DataFrame({'Total Purchase Value' : [tRevF, tRevM, tRevO]})
gender_purch_df["Total Purchase Value"] = TotalPurValGen

# Calc Average Total Purchase Price Per Person
gender_temp_df = purchase_data.loc[:, ["SN", "Gender","Price"]]
female_avg_pur_pri_df = gender_temp_df.loc[gender_temp_df["Gender"] == "Female"]
tPurValF = female_avg_pur_pri_df.groupby(['SN']).sum().mean()
gender_temp_df = purchase_data.loc[:, ["SN", "Gender","Price"]]
male_avg_pur_pri_df = gender_temp_df.loc[gender_temp_df["Gender"] == "Male"]
tPurValM = male_avg_pur_pri_df.groupby(['SN']).sum().mean()
gender_temp_df = purchase_data.loc[:, ["SN", "Gender","Price"]]
other_avg_pur_pri_df = gender_temp_df.loc[gender_temp_df["Gender"] == "Other / Non-Disclosed"]
tPurValO = other_avg_pur_pri_df.groupby(['SN']).sum().mean()
TotalPurPerUsr = pd.DataFrame({'Average Total Purchase Per Person' : [tPurValF[0], tPurValM[0], tPurValO[0]]})
gender_purch_df["Average Total Purchase Per Person"] = TotalPurPerUsr

# Clean up Data Sheet
gender_purch_df = gender_purch_df.drop(['Price'], axis=1)
gender_purch_df["Average Purchase Price"] = gender_purch_df["Average Purchase Price"].map("${:.2f}".format)
gender_purch_df["Total Purchase Value"] = gender_purch_df["Total Purchase Value"].map("${:.2f}".format)
gender_purch_df["Average Total Purchase Per Person"] = gender_purch_df["Average Total Purchase Per Person"].map("${:.2f}".format)
gender_purch_df = gender_purch_df.rename(columns={"SN": "Purchase Count"})
gender_purch_df = gender_purch_df.set_index("Gender")

gender_purch_df

,Purchase Count,Average Purchase Price,Total Purchase Value,Average Total Purchase Per Person
Gender,,,,
Female,113,$3.20,$361.94,$4.47
Male,652,$3.02,$1967.64,$4.07
Other / Non-Disclosed,15,$3.35,$50.19,$4.56


In [7]:
# Age
bins = [0, 9, 14, 19, 24, 29, 34, 39, purchase_data['Age'].max()]
binNames = ['<10', '10-14', '15-19', '20-24', '25-29', '30-34', '35-39', '40+']
age_base_df = purchase_data.copy()
age_demo_df = age_base_df.drop(["Item Name" ,'Item ID' ,"Gender" ,"Purchase ID", 'Price'], axis=1)
age_demo_df = age_demo_df.drop_duplicates(["SN"])
# df["Test Score Summary"] = pd.cut(df["Test Score"], bins, labels=group_names, include_lowest=True)
age_demo_df["Age Bracket"] = pd.cut(age_demo_df['Age'], bins, labels=binNames, include_lowest=True)
age_demo_df = age_demo_df.groupby(['Age Bracket']).agg({'SN' : 'count'})
age_demo_df['Percentage of Players'] = (age_demo_df["SN"] / totalNumPlayers * 100).map("{:.2f} %".format)
age_demo_df = age_demo_df.rename(columns={"SN": "Count"})
age_demo_df

,Count,Percentage of Players
Age Bracket,,
<10,17,2.95 %
10-14,22,3.82 %
15-19,107,18.58 %
20-24,258,44.79 %
25-29,77,13.37 %
30-34,52,9.03 %
35-39,31,5.38 %
40+,12,2.08 %


In [8]:
age_pur_df = age_base_df
age_pur_df = age_pur_df.drop(["Item Name" ,'Item ID' ,"Gender" ,"Purchase ID"], axis=1)
age_pur_df["Age Bracket"] = pd.cut(age_pur_df['Age'], bins, labels=binNames, include_lowest=True)
age_pur_df["Average Purchase Price"] = age_pur_df["Price"]
age_pur_df["Total Purchase Value"] = age_pur_df["Price"]
age_pur_df["Average Total Purchase per Person"] = age_pur_df["Price"]
age_pur_grup_df = age_pur_df.groupby(['Age Bracket'], as_index=False).agg({'SN' : 'count',
                                                                      'Average Purchase Price' : 'mean', 
                                                                      'Total Purchase Value' : 'sum',
                                                                      'Average Total Purchase per Person' : 'sum'}).copy()
age_avg_per_df = age_pur_df.copy()
ageATPpP_list = []
for x in range(len(binNames)):
    age_temp_df = age_avg_per_df.loc[age_avg_per_df["Age Bracket"] == binNames[x]]
    bracket_age_avg = age_temp_df.groupby(['SN']).sum().mean()
    ageATPpP_list.append(bracket_age_avg["Average Total Purchase per Person"])

age_pur_grup_df['Average Total Purchase per Person'] = ageATPpP_list

age_pur_grup_df["Average Purchase Price"] = age_pur_grup_df["Average Purchase Price"].map("${:.2f}".format)
age_pur_grup_df["Total Purchase Value"] = age_pur_grup_df["Total Purchase Value"].map("${:.2f}".format)
age_pur_grup_df["Average Total Purchase per Person"] = age_pur_grup_df["Average Total Purchase per Person"].map("${:.2f}".format)
age_pur_grup_df = age_pur_grup_df.rename(columns={"SN": "Purchase Count"})
age_pur_grup_df = age_pur_grup_df.set_index("Age Bracket")
age_pur_grup_df

,Purchase Count,Average Purchase Price,Total Purchase Value,Average Total Purchase per Person
Age Bracket,,,,
<10,23,$3.35,$77.13,$4.54
10-14,28,$2.96,$82.78,$3.76
15-19,136,$3.04,$412.89,$3.86
20-24,365,$3.05,$1114.06,$4.32
25-29,101,$2.90,$293.00,$3.81
30-34,73,$2.93,$214.00,$4.12
35-39,41,$3.60,$147.67,$4.76
40+,13,$2.94,$38.24,$3.19


In [9]:
top_spender_base_df = purchase_data.copy()

top_spend_temp_df = top_spender_base_df
top_spend_temp_df['Avg'] = top_spender_base_df['Price']
top_spend_temp_df = top_spender_base_df.groupby(["SN"], as_index=False).agg({'Purchase ID' : 'count', 'Price' : 'sum', 'Avg' : 'mean'})
top_spenders =  top_spend_temp_df.sort_values("Price", ascending=False).head()
columns = [{'Name', 'Purchase Count', 'Average Purchase Price', 'Total Purchase Value'}]
top_spenders = top_spenders.rename(columns={'SN' : 'Name',
                                            'Purchase ID' : 'Purchase Count',
                                            'Avg' : 'Average Purchase Price',
                                            'Price' : 'Total Purchase Value'})
top_spenders['Average Purchase Price'] = top_spenders['Average Purchase Price'].map("${:.2f}".format)
top_spenders['Total Purchase Value'] = top_spenders['Total Purchase Value'].map("${:.2f}".format)
top_spenders = top_spenders.reset_index(drop=True).set_index("Name")

top_spenders

,Purchase Count,Total Purchase Value,Average Purchase Price
Name,,,
Lisosia93,5,$18.96,$3.79
Idastidru52,4,$15.45,$3.86
Chamjask73,3,$13.83,$4.61
Iral74,4,$13.62,$3.40
Iskadarya95,3,$13.10,$4.37


In [10]:
pop_items_base_df = purchase_data.copy()
pop_items_df = pop_items_base_df.drop(['Purchase ID', 'SN', 'Age', 'Gender'], axis=1).copy()
pop_items_df['Purchase Count'] = pop_items_base_df['Item ID']
pop_items_df['Total Purchase Value'] = pop_items_base_df['Price']
pop_items_df = pop_items_df.rename(columns={'Price' : 'Item Price'})

pop_grup_df = pop_items_df.groupby(['Item ID'], as_index=False).agg({'Item Name' : 'max',
                                                                    'Item Price' : 'mean',
                                                                    'Purchase Count' : 'count',
                                                                    'Total Purchase Value' : 'sum'})
pop_sort_df = pop_grup_df.sort_values('Purchase Count', ascending=False)

pop_final_df = pop_sort_df.head().copy()

pop_final_df['Total Purchase Value'] = pop_final_df['Total Purchase Value'].map("${:.2f}".format)
pop_final_df['Item Price'] = pop_final_df['Item Price'].map("${:.2f}".format)
pop_final_df = pop_final_df.set_index(['Item ID', 'Item Name'])
pop_final_df

,,Item Price,Purchase Count,Total Purchase Value
Item ID,Item Name,,,
92,Final Critic,$4.61,13,$59.99
178,"Oathbreaker, Last Hope of the Breaking Storm",$4.23,12,$50.76
145,Fiery Glass Crusader,$4.58,9,$41.22
132,Persuasion,$3.22,9,$28.99
108,"Extraction, Quickblade Of Trembling Hands",$3.53,9,$31.77


In [11]:
prof_items_base_df = purchase_data.copy()
prof_items_df = prof_items_base_df.drop(['Purchase ID', 'SN', 'Age', 'Gender'], axis=1).copy()
prof_items_df['Purchase Count'] = prof_items_base_df['Item ID']
prof_items_df['Total Purchase Value'] = prof_items_base_df['Price']
prof_items_df = prof_items_df.rename(columns={'Price' : 'Item Price'})

prof_grup_df = prof_items_df.groupby(['Item ID'], as_index=False).agg({'Item Name' : 'max',
                                                                    'Item Price' : 'mean',
                                                                    'Purchase Count' : 'count',
                                                                    'Total Purchase Value' : 'sum'})
prof_sort_df = prof_grup_df.sort_values('Total Purchase Value', ascending=False)

prof_final_df = prof_sort_df.head().copy()

prof_final_df['Total Purchase Value'] = prof_final_df['Total Purchase Value'].map("${:.2f}".format)
prof_final_df['Item Price'] = prof_final_df['Item Price'].map("${:.2f}".format)
prof_final_df = prof_final_df.set_index(['Item ID', 'Item Name'])
prof_final_df

,,Item Price,Purchase Count,Total Purchase Value
Item ID,Item Name,,,
92,Final Critic,$4.61,13,$59.99
178,"Oathbreaker, Last Hope of the Breaking Storm",$4.23,12,$50.76
82,Nirvana,$4.90,9,$44.10
145,Fiery Glass Crusader,$4.58,9,$41.22
103,Singed Scalpel,$4.35,8,$34.80


In [12]:
# Everything below is me expermenting with making a summary .txt file
# This will output all the finished tables into a txt file

dfStrings = [totalNumPlayersDf.to_string(justify='center', index=False),
            PurSumDF.to_string(justify='center', index=False),
            gender_df.to_string(justify='center'),
            gender_purch_df.to_string(justify='center'),
            age_demo_df.to_string(justify='center'),
            age_pur_grup_df.to_string(justify='center'),
            top_spenders.to_string(justify='center'),
            pop_final_df.to_string(justify='center'),
            prof_final_df.to_string(justify='center')]

In [13]:
output_path = os.path.join("summary", "Heros_Data_Summary.txt")

# Checks if the directory exists, if it dosen't it makes one.
if not os.path.isdir("summary"):
    os.mkdir("summary")
else:
    pass

# Checks if path is valid. if it is, it creates a txt file containing the data
# else it gives as os error
try:
    with open(output_path, "w+") as dataOut:
        for df in dfStrings:
            dataOut.writelines(df + '\n\n\n')
            dataOut.writelines("------------------------------------------------------------------------------------------------------------\n")
except OSError:
    print("Creation of txt file failed")
else:
    pass